In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm
from matplotlib.colors import ListedColormap
from matplotlib import patches
from PIL import Image
import pandas as pd

In [ ]:
path = r''
save_path = r''
conc = 2 #mg/mL of fluorophore
font_size = 12
save_status = "" #"save" to save the PNG files
reference = "No" #"No" if no reference shall be taken, otherwise, index of image to be used as a reference
title_graph = ""

In [ ]:
def get_files(path):
    file_paths = [os.path.join(path,f) for f in sorted(os.listdir(path)) if f.endswith(".png")]
    filename =  [str(f) for f in sorted(os.listdir(path)) if f.endswith(".png")]
    return file_paths, filename

files = get_files(path)
def get_conc_maps(path, conc, font_size, save_status, ref):
    file_path, filename = get_files(path)
    if ref != "No":
        base_file = file_path[ref]
        pixel_basefile_G = get_pixels_RGB(base_file)
        max_pixelbf_G = np.max(pixel_basefile_G)
        k = conc/max_pixelbf_G
    if ref == "No":
        k = conc
    for fp, f in zip(file_path, filename):
        pixel_conc = plot_surface(fp, k, f, font_size, save_status)
        

def get_pixels_RGB(file):
    im = Image.open(file, 'r')
    pixel_RGB = np.array(im)
    pixel_G = np.float32(pixel_RGB[:,:,1])
    return (pixel_G)

def get_pixels_conc(pixel_G, k):
    max_pixel_G = np.max(pixel_G)
    if reference != "No":
        pixel_conc = (pixel_G * k)
    if reference == "No":
        pixel_conc = (pixel_G * k)/max_pixel_G
    return pixel_conc

def plot_surface(file, k, title, font_size, save_status):
    pixel_G = get_pixels_RGB(file)
    pixel_conc = get_pixels_conc(pixel_G, k)
    plt.figure(figsize=(6,6))
    cmap = cm['jet']
    newcolors = cmap(np.linspace(0, 1, 256))
    black = np.array([0, 0, 0, 1])
    newcolors[:12, :] = black
    newcmp = ListedColormap(newcolors)
    plt.imshow(pixel_conc, cmap = 'jet', vmax=conc, vmin=0)
    plt.colorbar().set_label(label="FITC concentration [mg/mL]", size=font_size)
    title =  " " + title[:-4]
    plot_format(title_graph + title, font_size, save_status)
    return pixel_conc

def plot_format(title, font_size, save_status):
    plt.title(title, size=font_size+2)
    plt.xlabel("Position [pixel]")
    plt.ylabel("Position [pixel]")
    plt.axis("off")
    if save_status == "save":
        plt.savefig(os.path.join(save_path, f"{title}.svg"), transparent=True)


In [ ]:
get_conc_maps(path, conc, font_size, save_status, reference)